# Examine Dudh Kosi ti-melt model inputs and results

In [ ]:
from __future__ import print_function
%pylab notebook
import datetime as dt
import glob
import matplotlib.pyplot as plt
import pdb
from charistools.hypsometry import Hypsometry
from charistools.meltModels import ImshowTriSurfMelt
from charistools.meltModels import PlotTriSurfInput
from charistools.meltModels import PlotTriSurfMelt
from charistools.modelEnv import ModelEnv
from charistools.timeSeries import TimeSeries
import pandas as pd
import re
import os

# Look at SOL input and melt for basin id 11066

In [ ]:
#on Summit:
#dir = '/work/charis/ti_model/derived_hypsometries/partner_basins/GA_DudhKosi_at_GokyoHydroID11066'
#on macice:
dir = '/Users/brodzik/modscag'
%cd $dir
%ls *2012*2strike*

nstrikes = 2
year = 2012
solHyps = Hypsometry('GA_DudhKosi_at_GokyoHydroID11066.2012.0100m.modicev04_2strike.snow_on_land_area_by_elev.txt')
solMeltHyps = Hypsometry('GA_DudhKosi_at_GokyoHydroID11066.2012.0100m.modicev04_2strike.snow_on_land_melt_by_elev.best_Karnali_SA_model.txt')


In [ ]:
solHyps.data

In [ ]:
sol_by_doy = solHyps.data_by_doy()
sol_melt_by_doy = solMeltHyps.data_by_doy()

In [ ]:
sol_by_doy

In [ ]:
sol_melt_by_month = sol_melt_by_doy.groupby([pd.TimeGrouper('M')]).sum().to_frame(name='melt')
yyyymm_index = pd.to_datetime(sol_melt_by_month.index.map(lambda x: x.strftime('%Y-%m-15')))
df = pd.DataFrame(data=sol_melt_by_month.values, index=yyyymm_index, columns=['melt'])
df['SOLmelt'] = sol_melt_by_doy.groupby([pd.TimeGrouper('M')]).sum().values
df
#sol_melt_by_month
    

In [ ]:
import matplotlib.dates as mdates

fig, ax = plt.subplots(3, 1, figsize=(10,6))
ax[0].plot(sol_by_doy)
ax[0].set_title('Dudh Kosi Basin Snow Covered Area 2012')
ax[0].set_ylabel('Area (' + r'$km^2$' + ')') 

ax[1].plot(sol_melt_by_doy)
ax[1].set_title('Dudh Kosi Basin SOL Modeled Melt Volume 2012')
ax[1].set_ylabel('Volume (' + r'$km^3$' + ')') 

start = dt.datetime(2012,1,1)
end = dt.datetime(2012,12,31)
dates_list = [start + dt.timedelta(days=x) for x in range(0, (end-start).days+1)]

myFmt = mdates.DateFormatter('%m/%d') 
months = mdates.MonthLocator()  
days = mdates.DayLocator(bymonthday=(1))   
dayss = mdates.DayLocator()

for i in np.arange(2):
    ax[i].set_xlim(dates_list[0],dates_list[-1])
    ax[i].xaxis.set_major_formatter(myFmt)
    ax[i].xaxis.set_major_locator(days)
    ax[i].xaxis.set_minor_locator(dayss)
    
ylim = np.amax(df["SOLmelt"])
ylim_min = 0.
df[["SOLmelt"]].plot(ax=ax[2], kind="bar",
                     edgecolor=(0.9, 0.9, 0.9))
ax[2].set_title("Monthly SOL Modeled Melt Volume 2012")
ax[2].set_ylabel('Volume (' + r'$km^3$' + ')') 
ax[2].xaxis.set_major_formatter(plt.NullFormatter())
#for container in ax[2].containers:
#    plt.setp(container, width=0.25)
ax[2].set_ylim([1.1 * ylim_min, 1.1 * ylim])
#ax[2].text(ax[2].get_xlim()[0] + (0.03 * (ax[2].get_xlim()[1] - ax[2].get_xlim()[0])),
#          ax[2].get_ylim()[1] * 0.7,
#          monthly_annotation,
#          style='italic',
#          bbox={'facecolor':'gray', 'alpha':0.1, 'pad':10})
#        handles, labels = ax[2].get_legend_handles_labels()
#ax[3].legend(loc='upper right')    

fig.tight_layout()
outfile = "basin_SOL_area_and_melt.2012.png"
fig.savefig(outfile)
print("Wrote melt_hyps to %s" % outfile)



In [ ]:
%pwd


In [ ]:
scag2015 = pd.read_csv('sca2015.txt', sep=' ')
scag2015 = scag2015.set_index('Date')
scag2015.index = pd.to_datetime(scag2015.index, format='%Y%m%d')
scag2015

In [ ]:
scag2016 = pd.read_csv('sca2016.txt', sep=' ')
scag2016 = scag2016.set_index('Date')
scag2016.index = pd.to_datetime(scag2016.index, format='%Y%m%d')
scag2016

In [ ]:
scag2012 = pd.read_csv('sca2012.txt', sep=' ')
scag2012 = scag2012.set_index('Date')
scag2012.index = pd.to_datetime(scag2012.index, format='%Y%m%d')
scag2012

In [ ]:
tmp2016 = scag2016.copy()
tmp2016

In [ ]:
tmp2016.index = tmp2016.index - pd.DateOffset(years=4)
tmp2016

In [ ]:
tmp2015 = scag2015.copy()
tmp2015.index = tmp2015.index - pd.DateOffset(years=3)
tmp2015

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))

ax.plot(tmp2016.SCA_km2, label='2016', color='green')
ax.plot(tmp2015.SCA_km2, label='2015', color='purple')
ax.plot(scag2012.SCA_km2, label='2012', color='orange')
ax.set_ylabel('Area (' + r'$km^2$' + ')') 

ax.set_title('Dudh Kosi Basin Snow Covered Area from scag')
ax.set_ylim([0., 1100.])

start = dt.datetime(2012,1,1)
end = dt.datetime(2012,12,31)
dates_list = [start + dt.timedelta(days=x) for x in range(0, (end-start).days+1)]

myFmt = mdates.DateFormatter('%m/%d') 
months = mdates.MonthLocator()  
days = mdates.DayLocator(bymonthday=(1))   
dayss = mdates.DayLocator()

for i in np.arange(2):
    ax.set_xlim(dates_list[0],dates_list[-1])
    ax.xaxis.set_major_formatter(myFmt)
    ax.xaxis.set_major_locator(days)
    ax.xaxis.set_minor_locator(dayss)

# myFmt = mdates.DateFormatter('%m/%d')
# ax.xaxis.set_major_formatter(myFmt)

ax.legend()
fig.tight_layout()
outfile = "scag2012_vs_2016.from_h25v06.png"
outfile = "scag2012_vs_2015_vs_2016.from_h25v06.png"
fig.savefig(outfile)
print("Wrote to %s" % outfile)


In [ ]:
%pwd
%ls *.mat


In [ ]:
from scipy.io import loadmat

# x = loadmat('test.mat')
# lon = x['lon']
# lat = x['lat']
## one-liner to read a single variable
# lon = loadmat('test.mat')['lon']

In [ ]:
def read_sca_from_matfile(filename):
    mdata = loadmat(filename)
    
    # Scale SCA array by 10000. to get fraction of pixel
    matSCA = mdata['Sc'] / 10000.
    RefMatrix = mdata['RefMatrix']
    
    # Return SCA and RefMatrix
    return(matSCA, RefMatrix)


In [ ]:
from charistools.readers import ModisTileCube


In [ ]:
def read_sca_from_cubefile(cubeFilename, doy):
    cube = ModisTileCube(cubeFilename, varname='fsca')
    cubeSCA = cube.read(doy=doy)
    cube.close()
    return(cubeSCA)

In [ ]:
def read_sca_data(mmdd_str, doy):
    
    matFile = '2012%s_h25v05.mat' % mmdd_str
    matSCA, RefMatrix = read_sca_from_matfile(matFile)
    
    cubeFile = 'MODSCAG_GF_Snow.v0.9.h25v05_2012.h5'
    cubeSCA = read_sca_from_cubefile(cubeFile, doy)
    
    fig, ax = plt.subplots(1,2,figsize=(8,4))

    ax[0].imshow(cubeSCA, vmin=0.0, vmax=1.0, interpolation='None')
    ax[0].set_title("SCAG from cube (orig) doy=%03d" % doy)
    ax[0].legend()

    ax[1].imshow(matSCA, vmin=0.0, vmax=1.0, interpolation='None')
    ax[1].set_title("SCAG from mat file (new) mmdd=%s" % mmdd_str)
    ax[1].legend()

    fig.tight_layout()
    outfile = "scag_origCube_vs_newMat.2012%s.png" % mmdd_str
    fig.savefig(outfile)
    print("Wrote to %s" % outfile)

    return(matSCA, cubeSCA, RefMatrix)

In [ ]:
doy = 1
read_sca_data('0101', doy)

In [ ]:
doy = 31 + 29 + 31 +30 + 15
read_sca_data('0515', doy)

In [ ]:
np.min(cdata), np.max(cdata)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,6))

ax[0].imshow(cdata, vmin=0.0, vmax=1.0, interpolation='None')
ax[0].set_title("SCAG from cube (orig)")
ax[0].legend()

ax[1].imshow(matSCA, vmin=0.0, vmax=1.0, interpolation='None')
ax[1].set_title("SCAG from mat file (new)")
ax[1].legend()

fig.tight_layout()
#outfile = baseFilename + label + '.melt_tseries.png'
#fig.savefig(outfile)
#print("Wrote melt_tseries to %s" % outfile)


In [ ]:
from charistools.readers import read_tile

In [ ]:
maskFile = 'GA_DudhKosi_at_GokyoHydroID11066.basin_mask.h25v06.tif'
maskFile


In [ ]:
mask = read_tile(maskFile)
mask.shape

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,6))

ax.imshow(mask, vmin=0.0, vmax=1.0, interpolation='None')
ax.set_title("Dudh Kosi mask")

fig.tight_layout()
#outfile = baseFilename + label + '.melt_tseries.png'
#fig.savefig(outfile)
#print("Wrote melt_tseries to %s" % outfile)